In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import joblib
from matplotlib import pyplot as plt
import MyNet

In [3]:
# path = '/home/zihan/codes/sgd-influence/experiment/Sec71/adult_dnn_up12/sgd000.dat'
path = '/home/zihan/codes/sgd-influence/experiment/Sec71/adult_dnn/sgd000.dat'
res = joblib.load(path)
print(res.keys())

/home/zihan/codes/sgd-influence/.venv/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


dict_keys(['models', 'info', 'counterfactual', 'alpha', 'main_losses', 'train_losses'])


In [6]:
# dict_keys(['models', 'info', 'counterfactual', 'alpha', 'main_losses', 'train_losses'])

# models: list of trained models
# type of models: <class 'MyNet.NetList'>
print(f"types of models: {type(res['models'])}")
print(f"length of models: {len(res['models'])}")

# info: dict of training information
# type of info: <class 'list'>
print(f"types of info: {type(res['info'])}")
print(f"length of info: {len(res['info'])}")
print(f"keys of info: {res['info'][0].keys()}")
# lens of 'idx' of info[0]
print(f"length of idx (batch size): {len(res['info'][0]['idx'])}")
# for now `ntr` is fixed to 200, steps in each epoch = 200/batch_size, epochs = len(info)/steps in each epoch
print(f"epochs: {len(res['info'])/(200/len(res['info'][0]['idx']))}")

# counterfactual: dict of counterfactual information
# type of counterfactual: <class 'List'>
print(f"types of counterfactual: {type(res['counterfactual'])}")
print(f"length of counterfactual: {len(res['counterfactual'])}")
# type of 'counterfactual'.models[0]: <class 'MyNet.NetList'>
print(f"type of counterfactual[0]: {type(res['counterfactual'][0])}")
print(f"length of counterfactual[0]: {len(res['counterfactual'][0])}")

# alpha: dict of alpha information
print(f"types of alpha: {type(res['alpha'])}") # types of alpha: <class 'float'>
print(f"alpha: {res['alpha']}")

# main_losses: dict of main_losses information
print(f"types of main_losses: {type(res['main_losses'])}") # <class 'List'>
print(f"length of main_losses: {len(res['main_losses'])}")

# train_losses: dict of train_losses information
print(f"types of train_losses: {type(res['train_losses'])}") # <class 'numpy.ndarray'>
print(f"shape of train_losses: {res['train_losses'].shape}")

types of models: <class 'MyNet.NetList'>
length of models: 121
types of info: <class 'list'>
length of info: 120
keys of info: dict_keys(['idx', 'lr'])
length of idx (batch size): 20
epochs: 12.0
types of counterfactual: <class 'list'>
length of counterfactual: 200
type of counterfactual[0]: <class 'MyNet.NetList'>
length of counterfactual[0]: 121
types of alpha: <class 'float'>
alpha: 0.001
types of main_losses: <class 'list'>
length of main_losses: 8
types of train_losses: <class 'numpy.ndarray'>
shape of train_losses: (121,)


In [30]:
path2 = '/home/zihan/codes/sgd-influence/experiment/Sec71/adult_dnn_up12/infl_true000.dat'
res2 = joblib.load(path2)
print(res2)

[-1.65000558e-03 -8.10980797e-04 -4.36902046e-04  5.69194555e-04
 -6.34968281e-04  4.48644161e-04  3.51130962e-04 -1.07246637e-03
 -3.16321850e-04  3.53884697e-03 -1.03700161e-03 -5.13821840e-04
 -2.35795975e-04  6.35683537e-04  7.68899918e-06 -2.23249197e-04
  3.16768885e-04 -3.98606062e-04 -3.80218029e-04 -4.01884317e-04
 -5.99026680e-04 -4.54527140e-03  1.41519308e-03 -4.46140766e-05
 -7.99924135e-04 -1.54122710e-03 -3.37064266e-03  7.66426325e-04
  4.22686338e-04  2.65747309e-04  1.65700912e-05 -1.38431787e-04
 -2.22921371e-05 -2.87681818e-04  1.39522552e-03  8.90046358e-04
 -1.85400248e-04 -6.03556633e-04  2.83360481e-04  6.18100166e-04
 -9.03308392e-05  9.11980867e-04  5.25236130e-04 -9.02563334e-04
  3.57687473e-04  5.85407019e-04 -1.16536021e-03 -9.87857580e-04
 -1.32748485e-03  4.44054604e-05  5.83529472e-05 -1.51365995e-04
 -7.00205564e-04 -2.36093998e-04 -1.49011612e-03  6.18696213e-05
 -2.54720449e-04 -1.66836381e-03  4.92125750e-04  9.59038734e-04
  5.43177128e-04 -5.15133